<a href="https://colab.research.google.com/github/HansHenseler/masdav2023/blob/main/Part_9_Exercises_Answers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analysing and visualising geolocation data

Part 9

Master of Advanced Studies in Digital Forensics & Cyber Investigation

Data Analytics and Visualization for Digital Forensics

(c) Hans Henseler, 2023

# Part 9

Importing libraries

In [ ]:
# Need to install various packages.
#
# Important library for many geopython libraries
!apt install gdal-bin python-gdal python3-gdal
# Install rtree - Geopandas requirment
!apt install python3-rtree
# Install Geopandas
!pip install geopandas
# Install Folium for Geographic data visualization
!pip install folium
# Install plotlyExpress
!pip install plotly_express

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Package python-gdal is not available, but is referred to by another package.
This may mean that the package is missing, has been obsoleted, or
is only available from another source
However the following packages replace it:
  gdal-bin

E: Package 'python-gdal' has no installation candidate
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libspatialindex-c6 libspatialindex-dev libspatialindex6
The following NEW packages will be installed:
  libspatialindex-c6 libspatialindex-dev libspatialindex6 python3-rtree
0 upgraded, 4 newly installed, 0 to remove and 16 not upgraded.
Need to get 365 kB of archives.
After this operation, 1,799 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libspatialindex6 amd64 1.9.3-2 [247 kB]
Get:2 http://archive

In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
import matplotlib
import matplotlib.pyplot as plt
import folium
import plotly_express as px

In [ ]:
!sudo apt-get install -y exiftool

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'libimage-exiftool-perl' instead of 'exiftool'
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  libarchive-zip-perl libmime-charset-perl libposix-strptime-perl libsombok3
  libunicode-linebreak-perl
Suggested packages:
  libencode-hanextra-perl libpod2-base-perl
The following NEW packages will be installed:
  libarchive-zip-perl libimage-exiftool-perl libmime-charset-perl
  libposix-strptime-perl libsombok3 libunicode-linebreak-perl
0 upgraded, 6 newly installed, 0 to remove and 20 not upgraded.
Need to get 3,395 kB of archives.
After this operation, 20.3 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libarchive-zip-perl all 1.60-1ubuntu0.1 [84.6 kB]
Get:2 http://archive.ub

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# Exercises

## 1 Analyse pictures from an Android 10 phone for geotags and plot on a map

In [1]:
# Download Android_10.zip from Digital Corpora (takes app 10 minutes to download in Google Colab)
#
!wget "https://digitalcorpora.s3.amazonaws.com/corpora/mobile/android_10/Android_10.zip"

--2023-08-25 08:13:22--  https://digitalcorpora.s3.amazonaws.com/corpora/mobile/android_10/Android_10.zip
Resolving digitalcorpora.s3.amazonaws.com (digitalcorpora.s3.amazonaws.com)... 52.92.146.161, 52.92.212.49, 52.92.192.153, ...
Connecting to digitalcorpora.s3.amazonaws.com (digitalcorpora.s3.amazonaws.com)|52.92.146.161|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10476724716 (9.8G) [application/zip]
Saving to: ‘Android_10.zip’

Android_10.zip      100%[===================>]   9.76G  13.7MB/s    in 11m 13s 

2023-08-25 08:24:36 (14.8 MB/s) - ‘Android_10.zip’ saved [10476724716/10476724716]



In [2]:
# In this zip there is a zip file containing a logical extraction of the file system
# /Android 10 Image with Documentation/Non-Cellebrite Extraction/Pixel 3.zip
# extract this zip file (takes about 4 minutes, use -j to ignore file path)

!unzip -j Android_10.zip  Android\ 10\ Image\ with\ Documentation/Non-Cellebrite\ Extraction/Pixel\ 3.zip

Archive:  Android_10.zip
  inflating: Pixel 3.zip             


In [18]:
# investigate the contents of this zip file for jpg files, extract, run the exiftool and plot on a map
# hint: use the unzip -l option (e.g. to find *.jpg)

!unzip -l "Pixel 3.zip" | grep jpg

    22062  2020-02-08 14:54   Pixel 3/data/data/com.enflick.android.TextNow/app_Leanplum_Documents/InApp_Countdown_Timer (2).jpg
    15012  2020-02-08 14:54   Pixel 3/data/data/com.enflick.android.TextNow/app_Leanplum_Documents/nonsale_avatar.jpg
   135034  2020-02-08 14:54   Pixel 3/data/data/com.enflick.android.TextNow/app_Leanplum_Documents/nonsale_defaultinterstitial_extra101.jpg
    41336  2020-01-31 21:15   Pixel 3/data/data/com.google.android.googlequicksearchbox/files/recently/thisisdfir@gmail.com-3692132869510698294.jpg
    13696  2020-01-31 21:14   Pixel 3/data/data/com.google.android.googlequicksearchbox/files/recently/thisisdfir@gmail.com-5812264224591469663.jpg
    29512  2020-01-31 21:13   Pixel 3/data/data/com.google.android.googlequicksearchbox/files/recently/thisisdfir@gmail.com-6982875141504110754.jpg
   577563  2020-01-29 11:55   Pixel 3/data/data/com.google.vr.apps.ornament/app_preview_temp/shared/com.google.vr.apps.ornament.content.stickers.spark/spark_preview.jpg


In [4]:
# Export all *.jpg and ignor paths (-j option) with destination  ./images (-d ./images).
# Note: choose N for None if unzip asks if it must overwrite a file.
#
!unzip -j -d ./images "Pixel 3.zip" "*.jpg"

Archive:  Pixel 3.zip
  inflating: ./images/InApp_Countdown_Timer (2).jpg  
  inflating: ./images/nonsale_avatar.jpg  
  inflating: ./images/nonsale_defaultinterstitial_extra101.jpg  
  inflating: ./images/thisisdfir@gmail.com-3692132869510698294.jpg  
  inflating: ./images/thisisdfir@gmail.com-5812264224591469663.jpg  
  inflating: ./images/thisisdfir@gmail.com-6982875141504110754.jpg  
  inflating: ./images/spark_preview.jpg  
 extracting: ./images/6BIqfFk.jpg    
 extracting: ./images/CXW0PEe.jpg    
  inflating: ./images/82150035d650b73619f729f8797a2979.jpg  
  inflating: ./images/043a8097f43c6fb006dd4a97ee9447be.jpg  
  inflating: ./images/04a860c62dd3c32bc987962bf50826f0.jpg  
  inflating: ./images/0cf03778a47919ffa242abbb61045546.jpg  
  inflating: ./images/15093b05575b105cfb87ca1f977ba155.jpg  
  inflating: ./images/235cf218066aa4e50fab6637df280536.jpg  
  inflating: ./images/2f8011301fcab6a4e63fd3d3f58bdf2e.jpg  
  inflating: ./images/40a02bf1a684b89c672336492d738f56.jpg  
  i

In [5]:
!sudo apt-get install -y exiftool

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'libimage-exiftool-perl' instead of 'exiftool'
The following additional packages will be installed:
  libarchive-zip-perl libmime-charset-perl libsombok3
  libunicode-linebreak-perl
Suggested packages:
  libposix-strptime-perl libencode-hanextra-perl libpod2-base-perl
The following NEW packages will be installed:
  libarchive-zip-perl libimage-exiftool-perl libmime-charset-perl libsombok3
  libunicode-linebreak-perl
0 upgraded, 5 newly installed, 0 to remove and 16 not upgraded.
Need to get 3,964 kB of archives.
After this operation, 23.5 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libarchive-zip-perl all 1.68-1 [90.2 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libimage-exiftool-perl all 12.40+dfsg-1 [3,717 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libmime-charset-perl all 1.012.2

In [7]:
!exiftool -csv -c '%+.6f' -gpslatitude -gpslongitude -GPSLatitudeRef -GPSLongitudeRef -r -ext jpg -T ./images > data.csv

In [19]:
!cat data.csv

SourceFile,GPSLatitude,GPSLongitude,GPSLatitudeRef,GPSLongitudeRef
./images/4.jpg,-,-,-,-
./images/cover-8544a5b2e95efd056b853be1907b60ad.jpg,-,-,-,-
./images/._thumbnail.jpg,-,-,-,-
./images/image-2020-02-08-15-35-37.jpg,-,-,-,-
./images/102.jpg,-,-,-,-
./images/r1920_10000999_cr_tail.jpg,-,-,-,-
./images/04a860c62dd3c32bc987962bf50826f0.jpg,-,-,-,-
./images/share-bg.321973c9.jpg,-,-,-,-
./images/c23d86bc56e4e24438b11e3570e3d65d.jpg,-,-,-,-
./images/101.jpg,-,-,-,-
./images/thisisdfir@gmail.com-3692132869510698294.jpg,-,-,-,-
./images/fbdc1910bb970ee446fc23a7fa5601be.jpg,-,-,-,-
./images/1581642305950.jpg,-,-,-,-
./images/f152cd39d46e6c1c2668422e32227c0e.jpg,-,-,-,-
./images/288_reduced.jpg,-,-,-,-
./images/r1920_darcula_bg_3_tail.jpg,-,-,-,-
./images/043a8097f43c6fb006dd4a97ee9447be.jpg,-,-,-,-
./images/295_reduced.jpg,-,-,-,-
./images/98.jpg,-,-,-,-
./images/thisisdfir@gmail.com-6982875141504110754.jpg,-,-,-,-
./images/7-6e2bf1bbb768daf2c68f218edefadd66.jpg,-,-,-,-
./images/3.jpg,-,

In [21]:
import pandas as pd
df = pd.read_csv('data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131 entries, 0 to 130
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   SourceFile       131 non-null    object
 1   GPSLatitude      131 non-null    object
 2   GPSLongitude     131 non-null    object
 3   GPSLatitudeRef   131 non-null    object
 4   GPSLongitudeRef  131 non-null    object
dtypes: object(5)
memory usage: 5.2+ KB


In [22]:
df

,SourceFile,GPSLatitude,GPSLongitude,GPSLatitudeRef,GPSLongitudeRef
0,./images/4.jpg,-,-,-,-
1,./images/cover-8544a5b2e95efd056b853be1907b60a...,-,-,-,-
2,./images/._thumbnail.jpg,-,-,-,-
3,./images/image-2020-02-08-15-35-37.jpg,-,-,-,-
4,./images/102.jpg,-,-,-,-
...,...,...,...,...,...
126,./images/8edb455844ab6c331dce82b714b8893b.jpg,-,-,-,-
127,./images/IMG-7f51747d682442822f56c78aea8b9b34-...,-,-,-,-
128,./images/235cf218066aa4e50fab6637df280536.jpg,-,-,-,-
129,./images/c589be27450a42e5da034da884c7d353.jpg,-,-,-,-


In [23]:
df = df[df.GPSLatitude !=  '-'].reset_index()
df

,index,SourceFile,GPSLatitude,GPSLongitude,GPSLatitudeRef,GPSLongitudeRef
0,50,./images/IMG_20200209_132353.jpg,+35.660981,-78.879897,North,West
1,56,./images/IMG_20200213_195030.jpg,+35.658867,-78.874217,North,West
2,100,./images/IMG_20190917_082637.jpg,+35.735647,-78.636367,North,West


In [24]:
import geopandas as gpd
jpg_gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.GPSLatitude, df.GPSLongitude))
jpg_gdf

,index,SourceFile,GPSLatitude,GPSLongitude,GPSLatitudeRef,GPSLongitudeRef,geometry
0,50,./images/IMG_20200209_132353.jpg,+35.660981,-78.879897,North,West,POINT (35.66098 -78.87990)
1,56,./images/IMG_20200213_195030.jpg,+35.658867,-78.874217,North,West,POINT (35.65887 -78.87422)
2,100,./images/IMG_20190917_082637.jpg,+35.735647,-78.636367,North,West,POINT (35.73565 -78.63637)


In [25]:
import folium


In [26]:
map = folium.Map(location = [35.660981,-78.879897,], tiles='OpenStreetMap' , width='75%',height='75%',zoom_start = 8)

# Create a geometry list from the GeoDataFrame
jpg_list = [[point.xy[0][0], point.xy[1][0]] for point in jpg_gdf.geometry ]

# Iterate through list and add a marker for each host in jpg_gdf

i = 0
for coordinates in jpg_list:
    map.add_child(folium.Marker(location = coordinates, popup=jpg_gdf.SourceFile[i]))
    i = i+1

map

## 2 Can you find any other sources besides jpg files that contain geo location and can you also extract  and visualise that information?

hint: consider sqlite files from Android apps

In [27]:
# One sqlite database is the myplaces db from Google maps
#
# gmm_myplaces.db
#

!unzip -l "Pixel 3.zip" | grep gmm_myplaces

    40960  2020-02-09 14:45   Pixel 3/data/data/com.google.android.apps.maps/databases/gmm_myplaces.db
    40960  2020-02-09 14:45   Pixel 3/sbin/.magisk/mirror/data/data/com.google.android.apps.maps/databases/gmm_myplaces.db


In [28]:
!unzip -j "Pixel 3.zip"  "Pixel 3/sbin/.magisk/mirror/data/data/com.google.android.apps.maps/databases/gmm_myplaces.db"

Archive:  Pixel 3.zip
  inflating: gmm_myplaces.db         


In [29]:
import sqlite3

gmmdb = sqlite3.connect('gmm_myplaces.db')

In [30]:
df = pd.read_sql_query("SELECT * from sync_item", gmmdb)
df

,corpus,key_string,timestamp,merge_key,feature_fprint,latitude,longitude,is_local,sync_item
0,8,1:0,1554407343604,0,None,35734602,-78636654,0,b'\x12\x031:02\xd1\x01\n\x04\x08\x01\x10\x00\x...


In [31]:
# need to divide by 1000000 because there is no decimal point

gmmdb_gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.latitude/1000000, df.longitude/1000000))
gmmdb_gdf

,corpus,key_string,timestamp,merge_key,feature_fprint,latitude,longitude,is_local,sync_item,geometry
0,8,1:0,1554407343604,0,None,35734602,-78636654,0,b'\x12\x031:02\xd1\x01\n\x04\x08\x01\x10\x00\x...,POINT (35.73460 -78.63665)


In [32]:
# the google timestamp is unix epoch in milliseconds. If we /1000 we can use datetime.fromtimestamp to get a datetime

from datetime import datetime
print(datetime.fromtimestamp(gmmdb_gdf.timestamp[0]/1000).strftime('Google %Y-%m-%d %H:%M:%S'))

Google 2019-04-04 19:49:03


In [33]:

map = folium.Map(location = [gmmdb_gdf.geometry[0].xy[0][0],gmmdb_gdf.geometry[0].xy[1][0]], tiles='OpenStreetMap' , width='75%',height='75%',zoom_start = 8)

# Create a geometry list from the GeoDataFrame
gmmdb_list = [[point.xy[0][0], point.xy[1][0]] for point in gmmdb_gdf.geometry ]

# Iterate through list and add a marker for each host in jpg_gdf
i = 0
for coordinates in gmmdb_list:
    popup = datetime.fromtimestamp(gmmdb_gdf.timestamp[i]/1000).strftime('Google myplaces timestamp %Y-%m-%d %H:%M:%S')
    map.add_child(folium.Marker(location = coordinates, popup=popup))
    i = i+1
map

In [34]:
# this location appears to be near the location of picture ./images/IMG_20190917_082637.jpg
# what does the exiftool tell us?

!exiftool ./images/IMG_20190917_082637.jpg

ExifTool Version Number         : 12.40
File Name                       : IMG_20190917_082637.jpg
Directory                       : ./images
File Size                       : 5.5 MiB
File Modification Date/Time     : 2020:01:31 21:22:58+00:00
File Access Date/Time           : 2023:08:25 08:51:42+00:00
File Inode Change Date/Time     : 2023:08:25 08:50:44+00:00
File Permissions                : -rw-r--r--
File Type                       : JPEG
File Type Extension             : jpg
MIME Type                       : image/jpeg
Exif Byte Order                 : Little-endian (Intel, II)
Make                            : Google
Camera Model Name               : Pixel 3
Orientation                     : Rotate 90 CW
X Resolution                    : 72
Y Resolution                    : 72
Resolution Unit                 : inches
Software                        : HDR+ 1.0.265696846zdh
Modify Date                     : 2019:09:17 08:26:37
Y Cb Cr Positioning             : Centered
Exposure Tim

In [35]:
# any other places? Here is a pdf file with a chapter on Android en Geo locations
# https://content-calpoly-edu.s3.amazonaws.com/cci/1/documents/ccic_forensics_manual/Android%20Forensics%20Chapter%209%20-%20Location%20Data.pdf
# but it could be an old manual

!unzip -l "Pixel 3.zip" | grep "misc/wifi"

        0  2020-02-15 10:22   Pixel 3/data/misc/wifi/
       43  2020-01-29 10:38   Pixel 3/data/misc/wifi/softap.conf
      626  2020-02-14 18:42   Pixel 3/data/misc/wifi/WifiConfigStore.xml
        0  2020-02-15 11:00   Pixel 3/sbin/.magisk/mirror/data/misc/wifi/
       43  2020-01-29 10:38   Pixel 3/sbin/.magisk/mirror/data/misc/wifi/softap.conf
      626  2020-02-14 18:42   Pixel 3/sbin/.magisk/mirror/data/misc/wifi/WifiConfigStore.xml


In [36]:
!unzip -j Pixel\ 3.zip Pixel\ 3/data/misc/wifi/WifiConfigStore.xml

Archive:  Pixel 3.zip
  inflating: WifiConfigStore.xml     


## 3 Cell tower visualisation

### 3a Download UMTS cell tower information from Swisscom and filter locations in Biel and store in new csv in your notebook

In [16]:
import folium
import pandas as pd

# Read the CSV file into a Pandas DataFrame (assuming the uploaded file name is 'xy_pwr_umts_170101.csv')
df_towers = pd.read_csv('xy_pwr_umts_170101.csv', sep=';')

# Center coordinates of Biel
biel_center_coordinates = "47.1381, 7.2467"

# Define the filtering boundaries for the 5 x 5 km area around the center  (approximately 0.045 degrees)
boundary_distance = 0.045

# Function to calculate distance between two sets of coordinates
def calculate_distance(coord1, coord2):
    lat1, lon1 = [float(val) for val in coord1.split(', ')]
    lat2, lon2 = [float(val) for val in coord2.split(', ')]
    return ((lat1 - lat2) ** 2 + (lon1 - lon2) ** 2) ** 0.5

# Filter the towers within the specified area
filtered_towers = df_towers[df_towers['Coordinates'].apply(
    lambda coord: calculate_distance(coord, biel_center_coordinates) <= boundary_distance
)]
filtered_towers


,ID,Coordinates,Geo Shape,Longitude,Latitude,Powercode,Powercode.1,Powercode.2,Powercode.3
69,103,"47.13671837451504, 7.255587869731409","{""coordinates"": [7.255587869731409, 47.1367183...",47.136718,7.255588,Mittel,Medium,Media,Moyenne
199,292,"47.12625746604414, 7.239518904572458","{""coordinates"": [7.239518904572458, 47.1262574...",47.126257,7.239519,Mittel,Medium,Media,Moyenne
228,331,"47.14807995070366, 7.245317585038221","{""coordinates"": [7.245317585038221, 47.1480799...",47.148080,7.245318,Mittel,Medium,Media,Moyenne
596,803,"47.13397077280427, 7.247543691180214","{""coordinates"": [7.247543691180214, 47.1339707...",47.133971,7.247544,Klein,Weak,Debole,Faible
898,1268,"47.1436971010196, 7.272412316656629","{""coordinates"": [7.272412316656629, 47.1436971...",47.143697,7.272412,Mittel,Medium,Media,Moyenne
...,...,...,...,...,...,...,...,...,...
11884,16216,"47.14898548488604, 7.278302556437657","{""coordinates"": [7.278302556437657, 47.1489854...",47.148985,7.278303,Klein,Weak,Debole,Faible
11932,16285,"47.12940287609875, 7.253925994539409","{""coordinates"": [7.253925994539409, 47.1294028...",47.129403,7.253926,Mittel,Medium,Media,Moyenne
11996,16367,"47.1187115351004, 7.250602872209965","{""coordinates"": [7.250602872209965, 47.1187115...",47.118712,7.250603,Mittel,Medium,Media,Moyenne
12043,16418,"47.140445950083304, 7.246874496030449","{""coordinates"": [7.246874496030449, 47.1404459...",47.140446,7.246874,Mittel,Medium,Media,Moyenne


In [17]:
# Create a map centered at Biel and zoomed in
m = folium.Map(location=[float(coord) for coord in biel_center_coordinates.split(', ')], zoom_start=13)

# Function to set circle radius based on power code
def get_circle_radius(power_code):
    if power_code.lower() == 'strong':
        return 10  # Adjust the radius as needed
    elif power_code.lower() == 'medium':
        return 7
    elif power_code.lower() == 'weak':
        return 5
    else:
        return 3

# Loop through the filtered towers and add circles to the map
for index, row in filtered_towers.iterrows():
    circle_radius = get_circle_radius(row['Powercode.1'])
    circle_color = 'red'
    circle_opacity = 0.5  # Adjust the opacity as needed

    folium.CircleMarker(
        location=[float(coord) for coord in row['Coordinates'].split(', ')],
        radius=circle_radius,
        color=circle_color,
        fill=True,
        fill_color=circle_color,
        fill_opacity=circle_opacity
    ).add_to(m)

# Display the map in the Colab notebook
m


### 3b Visualise the cell towers using Folium. Use the powercode column as a measure for the radius for a circle with the center at the cell tower location(make sensible assumptions on scale)

In [ ]:
# Your answer

### 3c Combine this view with the view presented in example 7 showing locations that were visited in Biel

In [ ]:
# Your answer